# Leer el archivo real
Primero, cargamos los datos como se hace en la vida real

In [1]:
# Importar librerias
import pandas as pd
# Leer el archivo desde la carpeta
data_ventas = pd.read_csv('files_folder/ventas_historico.csv')

# Ejemplo 1: El Resumen Básico (1 Dimensión)

El Caso: Finanzas quiere saber cuánto dinero bruto (ingresos) ha entrado por cada categoría de producto.

In [2]:
# Split: por 'categoria' | apply: columna 'ingresos_usd' | Combine: suma()
ingresos_por_categoria = data_ventas.groupby('categoria')['ingresos_usd'].sum()

print("--- Ingresos totales por categoria ---")
print(ingresos_por_categoria)

--- Ingresos totales por categoria ---
categoria
Accesorios     900
Laptops       7850
Software      1200
Name: ingresos_usd, dtype: int64


data_ventas: Llamamos a nuestra tabla completa.

.groupby('categoria'): Le decimos a Pandas: "Busca todos los valores diferentes que hay en la columna 'categoria' (Laptops, Accesorios, Software) y separa la tabla en 3 mini-tablas invisibles".

['ingresos_usd']: El Enfoque. Como las mini-tablas tienen muchas columnas (sucursal, vendedor, etc.), le decimos a Pandas: "Ignora el resto de columnas, solo quiero que mires los números que están en la columna de ingresos".

.sum(): ¡El Apply/Combine! Le da la orden final: "Suma todos esos números de cada mini-tabla y muéstrame el resultado pegado en una tablita nueva".

# Ejemplo 2: Creación de Variables + Agrupamiento Múltiple

El Caso: El Gerente de Operaciones dice: "Los ingresos no me dicen la rentabilidad. Necesito ver la ganancia neta (ingresos - costos) promedio, separada por Sucursal y por Categoría".

In [3]:
# 1. Primero creamos la columna de ganancia usando assign
data_rentabilidad = data_ventas.assign(
    ganancia_neta = lambda x:x['ingresos_usd']-x['costos_usd']
)

# 2. Ahora agrupamos por dos columnas (lista) y sacamos el promedio(mean)
ganancia_promedio =data_rentabilidad.groupby(['sucursal','categoria']) ['ganancia_neta'].mean()

print(" ---- Ganancia Promedio por Sucursal y producto----")
print(ganancia_promedio)

 ---- Ganancia Promedio por Sucursal y producto----
sucursal  categoria 
Bogota    Accesorios    110.0
          Laptops       525.0
          Software      250.0
CDMX      Accesorios    190.0
          Laptops       450.0
          Software      430.0
Madrid    Accesorios    220.0
          Laptops       700.0
          Software      340.0
Medellin  Accesorios    150.0
          Laptops       550.0
Name: ganancia_neta, dtype: float64


data_rentabilidad: Llamamos a la tabla que ya tiene la ganancia calculada.

.groupby(['sucursal', 'categoria']): Ojo a los corchetes [] internos. Le pasamos una lista de columnas. Pandas primero divide la tabla en ciudades (Bogotá, Medellín). Luego, dentro de la caja de Bogotá, vuelve a dividir por producto. Estás creando sub-grupos.

['ganancia_neta']: Nuevamente, le decimos qué columna numérica queremos analizar.

.mean(): En lugar de sumar todo, calcula el promedio matemático de esas ganancias en cada sub-grupo.

# Ejemplo 3: El Reporte de Recursos Humanos (.agg)

El Caso: HR va a dar los bonos del mes. Necesitan evaluar a cada vendedor. Te piden 3 métricas de un solo golpe: cuántas ventas hizo, cuánto dinero trajo en total y cuál fue su venta más cara.

In [4]:
# Usamos .agg para hacer 3 matemáticas distintas sobre columnas distintas
reporte_vendedores = data_ventas.groupby('vendedor').agg(
    total_transacciones = ('id_venta', 'count'),    # Cuenta cuántos IDs hay
    dinero_traido = ('ingresos_usd', 'sum'), #Suma el dinero
    Venta_Mas_alta = ('ingresos_usd', 'max')    #Busca el número mayor
)

print("--- Reporte de Vendedores----")
print(reporte_vendedores)

--- Reporte de Vendedores----
          total_transacciones  dinero_traido  Venta_Mas_alta
vendedor                                                    
Ana                         3           3350            1550
Carlos                      2           1650            1400
Jorge                       1            150             150
Luis                        2           1800            1600
Marta                       1            500             500
Pedro                       1            300             300
Sofia                       2           2200            1800


.groupby('vendedor'): Divide la tabla en mini-tablas, una por cada vendedor (Ana, Luis, Carlos...).

.agg(...): Significa "Aggregation" (Agregación). Es una función especial que le dice a Pandas: "Prepárate, porque no te voy a pedir una sola matemática, te voy a pedir varias sobre columnas diferentes".

Total_Transacciones = : Este es el nombre que tú inventas para la nueva columna del reporte.

('id_venta', 'count'): Es una tupla (paréntesis) con dos instrucciones. Primero, mira la columna 'id_venta'. Segundo, aplícale la matemática 'count' (contar cuántas filas hay). Nota que aquí la matemática va entre comillas.

 Dinero_Traido = ('ingresos_usd', 'sum'): Crea otra columna, mira los ingresos y los suma ('sum').

# Ejemplo 4: El Pipeline Senior (Filtro + Agrupación + Orden)

El Caso: El equipo de Marketing hizo una alianza con Visa/Mastercard. Quieren saber qué Sucursal es la que más ingresos genera, pero únicamente contando las compras hechas con "Tarjeta". Quieren ver el resultado ordenado del mejor al peor.

In [5]:
# Encadenamos todo entre paréntesis para que se lea como un libro
top_sucursales_tarjeta = (
    data_ventas
    .query("metodo_pago == 'Tarjeta'") # 1. Filtramos: Solo Tarjeta
    .groupby('sucursal')    # 2. Agrupamos por sucursal
    ['ingresos_usd'].sum()  # 3. Sumamos los ingresos
    .sort_values(ascending=False)   # 4. Ordenamos de mayor a menor
)

print("--- TOP Sucursales (Pago con tarjeta)---")
print(top_sucursales_tarjeta)

--- TOP Sucursales (Pago con tarjeta)---
sucursal
Bogota    3050
CDMX      1650
Madrid     700
Name: ingresos_usd, dtype: int64


(  ) externos: Sirven para engañar a Python. Le dicen: "Todo esto es una sola línea de código, pero lo voy a escribir con saltos de línea (Enter) para que sea hermoso de leer".

.query("metodo_pago == 'Tarjeta'"): ¡El filtro! Antes de hacer cualquier matemática, le decimos que elimine temporalmente todas las filas que dicen "Efectivo" o "Transferencia".

.groupby('sucursal')['ingresos_usd'].sum(): Toma esa tabla ya filtrada, la rompe por ciudad, se enfoca en el dinero y lo suma.

.sort_values(ascending=False): Toma el resultado de esa suma y lo ordena. ascending=False significa que lo ordene de forma Descendente (del más grande al más pequeño), para que el "ganador" quede en la fila número 1.